In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('./npr.csv')

In [3]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [6]:
# DOCUMENT TERM MATRIX (DTM)
dtm = cv.fit_transform(npr['Article'])

In [9]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
lda = LatentDirichletAllocation(n_components=7, verbose=1, random_state=42)

In [12]:
lda.fit(dtm)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(n_components=7, random_state=42, verbose=1)

In [16]:
# grab vocab word
cv.get_feature_names_out()[15000]

'dominican'

In [17]:
type(cv.get_feature_names_out())

numpy.ndarray

In [28]:
# grab the topics
# len(lda.components_) ---> 7
single_topic = lda.components_[0]
top_words = single_topic.argsort()[-20:]
for index in top_words:
    print(cv.get_feature_names_out()[index])

president
state
tax
insurance
trump
companies
money
year
federal
000
new
percent
government
company
million
care
people
health
said
says


In [27]:
# get highest probability words per topic
for i, topic in enumerate(lda.components_):
    print(f"Top 20 words for topic {i}")
    print([cv.get_feature_names_out()[index] for index in topic.argsort()[-20:]])
    print('\n')

Top 20 words for topic 0
['president', 'state', 'tax', 'insurance', 'trump', 'companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


Top 20 words for topic 1
['white', 'according', 'attack', 'reported', 'war', 'military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


Top 20 words for topic 2
['little', 'know', 'don', 'year', 'make', 'way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


Top 20 words for topic 3
['world', 'research', 'university', 'percent', 'care', 'time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


Top 20 words for topic 4
['donald', 'political', 'states', 'law', 'just', 'voters', 'vote', 'election', 'party', 'new', 'obama', 'cou

In [29]:
topic_result = lda.transform(dtm)

In [35]:
topic_result[9584].round(2)

array([0.  , 0.  , 0.62, 0.15, 0.  , 0.17, 0.06])

In [37]:
topic_result[9584].round(2).argmax()

2

In [38]:
npr['topic_index'] = topic_result.argmax(axis=1)

In [39]:
npr

,Article,topic_index
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
...,...,...
11987,The number of law enforcement officers shot an...,1
11988,"Trump is busy these days with victory tours,...",4
11989,It’s always interesting for the Goats and Soda...,3
11990,The election of Donald Trump was a surprise to...,4


In [42]:
import numpy as np
topic_names = {
    0: "Politics and Economy",
    1: "National Security and Foreign Policy",
    2: "Social and Cultural Life",
    3: "Health and Science",
    4: "Political Campaigns and Ideologies",
    5: "Arts and Personal Development",
    6: "Education and Development"
}

max_topic_indices = np.argmax(topic_result, axis=1)

npr['topic'] = [topic_names[index] for index in max_topic_indices]

In [43]:
npr

,Article,topic_index,topic
0,"In the Washington of 2016, even when the polic...",1,National Security and Foreign Policy
1,Donald Trump has used Twitter — his prefe...,1,National Security and Foreign Policy
2,Donald Trump is unabashedly praising Russian...,1,National Security and Foreign Policy
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1,National Security and Foreign Policy
4,"From photography, illustration and video, to d...",2,Social and Cultural Life
...,...,...,...
11987,The number of law enforcement officers shot an...,1,National Security and Foreign Policy
11988,"Trump is busy these days with victory tours,...",4,Political Campaigns and Ideologies
11989,It’s always interesting for the Goats and Soda...,3,Health and Science
11990,The election of Donald Trump was a surprise to...,4,Political Campaigns and Ideologies
